In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

In [3]:
import mxnet as mx

In [4]:
train_X = mx.nd.array(df_train.drop(["vendor_id", "payment_type", "fare_amount"], axis=1).values)
train_y = mx.nd.array(df_train.fare_amount.values)
test_X = mx.nd.array(df_test.drop(["vendor_id", "payment_type", "fare_amount"], axis=1).values)
test_y = mx.nd.array(df_test.fare_amount.values)
train_nd = list(zip(train_X, train_y))
test_nd = list(zip(test_X, test_y))

In [5]:
from mxnet.gluon.data import DataLoader

In [6]:
batch_size = 64
mx.random.seed(42)

In [7]:
train_data = DataLoader(train_nd, batch_size, shuffle=True)
test_data = DataLoader(test_nd, batch_size, shuffle=True)

In [8]:
from mxnet.gluon.nn import HybridSequential, Dense, Dropout
from mxnet.initializer import Xavier
from mxnet.gluon.loss import L2Loss
from mxnet.gluon.trainer import Trainer

In [9]:
net = HybridSequential()
with net.name_scope():
    net.add(Dense(9))
    net.add(Dropout(.25))
    net.add(Dense(16))
    net.add(Dropout(.25))
    net.add(Dense(1))

In [10]:
net.hybridize()

In [11]:
ctx = mx.cpu()

In [12]:
net.collect_params().initialize(Xavier(magnitude=2.24), ctx=ctx)

In [13]:
loss = L2Loss()

In [14]:
trainer = Trainer(net.collect_params(), optimizer="adam")

In [15]:
smoothing_constant = .01
epochs = 5

In [16]:
def measure_performance(model, ctx, data_iter):
    mae = mx.metric.MAE()
    for _, (data, labels) in enumerate(data_iter):
        data = data.as_in_context(ctx)
        labels = labels.as_in_context(ctx)
        output = model(data)
        predictions = output
        mae.update(preds=predictions, labels=labels)
    return mae.get()[1]

In [17]:
from mxnet import autograd

In [18]:
for e in range(epochs):
    moving_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data)
            loss_result = loss(output, label)
        loss_result.backward()
        trainer.step(batch_size)

        curr_loss = mx.nd.mean(loss_result).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + smoothing_constant * curr_loss)

    test_mae = measure_performance(net, ctx, test_data)
    train_mae = measure_performance(net, ctx, train_data)
    print("Epoch %s. Loss: %s, Train_mae %s, Test_mae %s" % (e, moving_loss, train_mae, test_mae))

Epoch 0. Loss: 14.677414258850465, Train_mae 1.7176581395542598, Test_mae 1.7429702549209596
Epoch 1. Loss: 9.018840810018597, Train_mae 1.008236643350756, Test_mae 1.027883416727066
Epoch 2. Loss: 8.634886879483181, Train_mae 0.902963298600298, Test_mae 0.9227513971195221
Epoch 3. Loss: 8.103130790031337, Train_mae 1.7556065334865707, Test_mae 1.7794139747543336
Epoch 4. Loss: 7.944741475106001, Train_mae 0.8874958783126203, Test_mae 0.9068851193103791


In [19]:
import numpy as np

In [20]:
net(mx.nd.array(np.expand_dims(np.ones(4), axis=0))).asnumpy().ravel().tolist()[0]

4.389521598815918

In [21]:
net.export("../models/model", epoch=epochs - 1)